In [1]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

- **使用具有结构化输出的聊天模型将文本分类到类别或标签中。** Classify text into categories or labels using chat models with structured outputs.

# 自定义结构化输出类

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama


tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = ChatOllama(
    model="qwen2.5:latest",
    temperature=0.7,
    # other params...
).with_structured_output(
    Classification
)
llm

RunnableBinding(bound=ChatOllama(model='qwen2.5:latest', temperature=0.7), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'description': 'The sentiment of the text', 'type': 'string'}, 'aggressiveness': {'description': 'How aggressive the text is on a scale from 1 to 10', 'type': 'integer'}, 'language': {'description': 'The language the text is written in', 'type': 'string'}}, 'required': ['sentiment', 'aggressiveness', 'language'], 'type': 'object'}}}], 'structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'description': 'The sentiment of the text', 'type': 'string'}, 'aggressiveness': {'description': 'How aggressive the text is on a scale from 1 to 10', 'type': 'integer'}, 'language': {'description': 'The language the text is written in', 'type

In [3]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='spanish')

## 输出字典格式

In [4]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': 'negative', 'aggressiveness': 8, 'language': 'spanish'}

# 更精细控制自定义的结构化输出

In [5]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [6]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOllama(
    model="qwen2.5:latest",
    temperature=0.7,
    # other params...
).with_structured_output(
    Classification
)
llm

RunnableBinding(bound=ChatOllama(model='qwen2.5:latest', temperature=0.7), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'enum': ['happy', 'neutral', 'sad'], 'type': 'string'}, 'aggressiveness': {'description': 'describes how aggressive the statement is, the higher the number the more aggressive', 'enum': [1, 2, 3, 4, 5], 'type': 'integer'}, 'language': {'enum': ['spanish', 'english', 'french', 'german', 'italian'], 'type': 'string'}}, 'required': ['sentiment', 'aggressiveness', 'language'], 'type': 'object'}}}], 'structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Classification', 'description': '', 'parameters': {'properties': {'sentiment': {'enum': ['happy', 'neutral', 'sad'], 'type': 'string'}, 'aggressiveness': {'description': 'describes how aggressive the statement is, the higher the number the more aggressive', 'enu

In [7]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='positive', aggressiveness=0, language='Spanish')

In [8]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='anger', aggressiveness=8, language='es')

In [9]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=0, language='en')